テンソル：Tensors 
==========================

テンソルは特殊なデータ構造で、配列や行列によく似ています。

PyTorchではテンソル型の変数を使用して、モデルの入力と出力、そしてモデルのパラメータを表現します。

テンソルは[NumPy](https://numpy.org/)のndarraysに似ていますが、違いとしてGPUや他のハードウェアアクセラレータ上で動作させることができます。

テンソルとNumPyの配列は基本的には同じメモリを共有することができるため、2つの型間での変換時にはデータをコピーする必要がありません。


テンソルはその他に、自動微分に最適化されています（この点については、後ほど5. 自動微分にて、詳しく説明します）。

NumPyのndarraysに慣れている人は、Tensor APIをすぐに使いこなせると思います。

そうでない場合には、本チュートリアルを通してぜひ習得してください。

In [2]:
%matplotlib inline

In [3]:
import torch
import numpy as np

テンソルの初期化
=======================

テンソルは様々な手法で初期化できます。

以下に例を示します。

**データから直接テンソルに変換**

データから直接テンソルを作ることができます。

その際、データ型は自動的に推測されます。

In [5]:
data = [[1,2],[3,4]]
x_data = torch.tensor(data)
x_data

tensor([[1, 2],
        [3, 4]])

**NumPy arrayからテンソルに変換**

テンソルとNumpy arraysは相互に変換可能です。


In [7]:
np_array = np.array(data)
x_np = torch.from_numpy(np_array)
x_np

tensor([[1, 2],
        [3, 4]])

**他のテンソルから作成**

他のテンソルから新しいテンソルを作成する場合、明示的に上書きされない限り、引数のテンソルのプロパティ（形状、データ型）を保持します。


In [8]:
x_ones = torch.ones_like(x_data) # x_dataの特性を維持
print(f"Ones Tensor: \n {x_ones} \n")

x_rand = torch.rand_like(x_data, dtype=torch.float) # x_dataのdatatypeを上書き
print(f"Random Tensor: \n {x_rand} \n")

Ones Tensor: 
 tensor([[1, 1],
        [1, 1]]) 

Random Tensor: 
 tensor([[0.6551, 0.2417],
        [0.5058, 0.5717]]) 



**ランダム値や定数のテンソルの作成**


``shape``は、テンソルの次元を示すタプルです。

以下の例では、shapeからテンソルのサイズを決めています。



In [9]:
shape = (2,3,)
rand_tensor  = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"Random Tensor: \n {rand_tensor} \n")
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Zeros Tensor: \n {zeros_tensor} \n")


Random Tensor: 
 tensor([[0.6993, 0.9583, 0.7754],
        [0.6162, 0.9553, 0.1885]]) 

Ones Tensor: 
 tensor([[1., 1., 1.],
        [1., 1., 1.]]) 

Zeros Tensor: 
 tensor([[0., 0., 0.],
        [0., 0., 0.]]) 



テンソルの操作
==================

PyTorchでは、算術、線形代数、行列操作（転置、インデックス、スライス）など、100種類以上のテンソル演算が可能です。

種々操作の詳細は[こちら]((https://pytorch.org/docs/stable/torch.html)
)をご覧ください。


各操作はGPU上で実行可能です（一般的にCPUの場合より高速です）。


Google Colaboratoryを使用している場合は、GPUを使用できるように設定してください。

（Runtime > Change runtime type > GPU）

デフォルトではテンソルはCPU上で作られます。

そのため、明示的に、``.to``メソッドを使用して、テンソルをGPU上へと移動させます。

大きなテンソルをデバイス間でコピーすると、時間とメモリの面でコストがかかる点にご注意ください。


In [14]:
# GPUが使用可能であれば、GPU上にテンソルを移動させる
tensor = torch.ones(4, 4)
if torch.cuda.is_available():
  tensor = tensor.to('cuda')
tensor

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], device='cuda:0')

テンソル操作の中からいくつかを試してみましょう。

NumPy APIに慣れていれば、Tensor APIも簡単に使えるようになると思います。

**numpy-likeなindexingとslicing:**

In [16]:
tensor = torch.ones(4, 4)
print('First row: ',tensor[0])
print('First column: ', tensor[:, 0])
print('Last column:', tensor[..., -1])
tensor[:,1] = 0
print(tensor)

First row:  tensor([1., 1., 1., 1.])
First column:  tensor([1., 1., 1., 1.])
Last column: tensor([1., 1., 1., 1.])
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])


- `:` は特定の次元をスライスするために使う。
- `...` は任意の数の次元をスキップするために使い、特に高次元テンソルで便利。

例えば、3次元テンソルやそれ以上の次元のテンソルを扱うとき、`...`を使うことで簡潔に指定できるんや：

**テンソルの結合** 


``torch.cat``を使用することで、テンソルを特定の次元に沿って結合させることができます（詳細は[こちら](https://pytorch.org/docs/stable/generated/torch.stack.html)をご覧ください）。

``torch.cat``とは微妙に異なるテンソル結合演算である[``torch.stack``](https://pytorch.org/docs/stable/generated/torch.stack.html)も確認しておいてください。


In [18]:
import torch

tensor1 = torch.tensor([[1, 2], [3, 4]])
tensor2 = torch.tensor([[5, 6], [7, 8]])

# torch.catの場合
cat_result = torch.cat((tensor1, tensor2), dim=0)
print('torch.cat result:\n', cat_result)

# torch.stackの場合
stack_result = torch.stack((tensor1, tensor2), dim=0)
print('torch.stack result:\n', stack_result)


torch.cat result:
 tensor([[1, 2],
        [3, 4],
        [5, 6],
        [7, 8]])
torch.stack result:
 tensor([[[1, 2],
         [3, 4]],

        [[5, 6],
         [7, 8]]])


**算術演算**


In [19]:
# 2つのテンソル行列のかけ算です。 y1, y2, y3 は同じ結果になります。
y1 = tensor @ tensor.T
y2 = tensor.matmul(tensor.T)

y3 = torch.rand_like(tensor)
torch.matmul(tensor, tensor.T, out=y3)


# こちらは、要素ごとの積を求めます。 z1, z2, z3 は同じ値になります。
z1 = tensor * tensor
z2 = tensor.mul(tensor)

z3 = torch.rand_like(tensor)
torch.mul(tensor, tensor, out=z3)

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])

**1要素のテンソル** 

1要素のテンソル（テンソルの全要素を足し算する等をした結果生まれます）を扱う場合には、``.item()``を使用することでPythonの数値型変数に変換できます。


In [20]:
agg = tensor.sum()
agg_item = agg.item()  
print(agg_item, type(agg_item))

12.0 <class 'float'>


**インプレース操作**


演算結果をオペランドに格納する演算をインプレースと呼びます。

メソッドの最後、接尾辞として操作名に、 ``_`` が付きます。

例えば、``x.copy_(y)``, ``x.t_()``であり、``x``の内容そのものを更新します。

In [21]:
print(tensor, "\n")
tensor.add_(5)
print(tensor)

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]]) 

tensor([[6., 5., 6., 6.],
        [6., 5., 6., 6.],
        [6., 5., 6., 6.],
        [6., 5., 6., 6.]])


【注意】


インプレース操作はメモリを節約できますが、演算履歴が失われてしまうため、微分を計算する際には問題となります。

そのため、そのような微分を求める場面ではインプレース操作の使用は推奨されていません。

NumPyとの変換
=========================

CPU上のテンソルとNumpy arraysは同じメモリを共有することができ、相互変換が容易です。

**Tensor to NumPy array**

In [22]:
t = torch.ones(5)
print(f"t: {t}")
n = t.numpy()
print(f"n: {n}")

t: tensor([1., 1., 1., 1., 1.])
n: [1. 1. 1. 1. 1.]


この際、テンソルが変化すると、Numpy側も変化します。


In [23]:
t.add_(1)
print(f"t: {t}")
print(f"n: {n}")

t: tensor([2., 2., 2., 2., 2.])
n: [2. 2. 2. 2. 2.]


**NumPy array to Tensor**


In [25]:
n = np.ones(5)
t = torch.from_numpy(n)

NumPy arrayの変化はテンソル側にも反映されます。

In [26]:
np.add(n, 1, out=n)
print(f"t: {t}")
print(f"n: {n}")

t: tensor([2., 2., 2., 2., 2.], dtype=torch.float64)
n: [2. 2. 2. 2. 2.]
